### Similar Strings

Jimmy loves playing with strings. He thinks string $A$ is similar to string $B$ if the following conditions are satisfied:

- Both strings have the same length (i.e., $A = a_0 a_1 ... a_{n-1}$  and $B = b_0 b_1 ... b_{n-1}$).
- For each valid pair of indices,$(i,j)$, in the strings, $[a_i = a_j$ and $b_i = b_j]$ or $[a_i \ne a_j$ and $b_i \ne b_j]$.

He has string, $S$, size of $n$ and gives you $q$ queries to answer where each query is in the form of a pair of integers $(i_i, r_i)$. For each substring $S[l_i, r_i]$, find the number of substrings $S[x,y]$ where substring $S[l_i, r_i]$ is similar to substring $S[x,y]$ and print this number on a new line.

Note: Substring $S[x,y]$ is the contiguous sequence of characters from index $x$ to index $y$. For example, if $S=$ `abcdefgh`, then $S[3,6]=$ `cdef`.

**Input Format**

The first line contains two space-separated integers describing the respective values of $n$ and $q$. 
The second line contains string $S$. 
Each line $i$ of the $q$ subsequent lines contains two space-separated integers describing the respective values of $l_i$ and $r_i$ for query i.

**Constraints**
- $1\le n,q\le 5 \times 10^4$
- $1\le L_i\le R_i \le n$
- $s_i \in {a,b,c,d,e,f,g,h,i,j}$

**Output Format**

For each query, print the number of similar substrings on a new line.

**Sample Input**
```
8 4
giggabaj
1 1
1 2
1 3
2 4
```

**Sample Output**
```
8
6
2
1
```

**Explanation**

We perform the following sequence of queries:

1. Strings with length $1$ are all similar, so our answer is $8$.
2. `gi`, `ig`, `ga`, `ab`, `ba`, and `aj` are similar, so our answer is $6$.
3. `gig` and `aba` are similar, so our answer is $2$.
4. `igg` has no similar string, so our answer is $1$.


### Naive Implementation
1. Init $counter=0$.
2. Generate all substrings of $S$ and store them in a dictionary $dict$, with $dict[l]$ holding the list of substrings with length $l$.
3. Compute the length of the query $q$.
4. Iterate over all substrings that have the length of the query.
5. For $q$ and for each substring generate a $signature$, based on the above conditions as follows:
```
For example "ggi":
        1st 2nd 3rd
1st      ==  !=
2nd          !=
3rd
```
    Notice the symmetry of the table.
6. Compare the $signatures$ and increment the $counter$ on equality.

In [ ]:
import sys
from collections import defaultdict

"""
Create a dictionary of lists. dict[l] holds a list with all the substrings of str that have
length l.
"""
def getAllSubstrings(str):
	dict = defaultdict(list)
	length = len(str)
	for i in range(length):
		for j in range(i,length):
			subStringLen = j - i + 1
			dict[subStringLen].append(str[i:j+1])
	return dict

"""
For example "ggi":
    1st 2nd 3rd
1st      ==  !=
2nd          !=
3rd
"""
def signature(str):
	if (str in signatures):
		return signatures[str]
	
	length = len(str)
	sig = [[False]*(length) for i in range(length)]
	for i in range(0, length):
		sig[i][i] = True
		
	for i in range(0, length):
		for j in range(i+1, length):
			sig[i][j] = (str[i] == str[j])
			sig[j][i] = sig[i][j]
	
	signatures[str]	= sig
	return sig

"""
Return true if str has the same signature as targetSig. Returns False otherwise
"""	
def isSimilar(str, targetSig):
	length = len(str)
	strSig = signature(str)
	for i in range(0, length):
		for j in range(i+1, length):
			if (strSig[i][j] != targetSig[i][j]):
				return False
	return True
	
	
def solve(l,r,s,subStrings):
    target = s[l-1:r]
    targetSig = signature(target)
    targetLength = len(target)
    subStrCandidates =  subStrings[targetLength]
    count = 0
    for str in subStrCandidates:
        if (isSimilar(str, targetSig)):
            count += 1
	
    return count
	
    
def main():
	f = open("in.txt")
	n, queries = (int(i) for i in f.readline().split())
	s = f.readline().rstrip()
	subStrings = getAllSubstrings(s)
	for q in range(0, queries):
		l, r = (int(i) for i in f.readline().split())        
		print(solve(l,r,s,subStrings))


		
signatures = {}
main()	

### Optimization
The above algorithm still has room for improvement.

#### Build a faster isSimilar function 
We can do that by _normalizing_ our Strings first before comparing. Example `gigxx` to `abacc`:

**Algorithm:**
1. Init dictionary `changedChars` with `changedChars[c] = c'`, that is, character `c` has been replaced with character `c'`.
2. Iterate over all characters of the input string:
	- If one character has not been changed before (and thus not contained in `changedChars`), change it to the next available char (`a,b,c,d,...`)
	- If we have changed the character before, reuse the replacement character.
	
**Example:**

Input `gigxx`

- First character is `g`. Change `g` -> `a`, since it is the first character we are changing, we start with `a`. All follow up `g` will be transformed to an `a`
- Next, is `i`. Since we haven't changed an `i` before, we change it to `b`.
- Next, is `g`. Since we already transformed a `g` before, we change this one also to `a`.
- Next, is `x`. Since we haven't changed an `x` before, we change it to `c`.
- Next, is another `x`. Since we already transformed an `x` before, we change this one also to `c`.

Result: `abacc`



In [ ]:
import sys
from collections import defaultdict

"""
Create a dictionary of lists. map[i] holds a list with all the substrings of str that have
length i.
"""
def getAllSubstrings(str):
	map = defaultdict(list)
	length = len(str)
	for i in range(length):
		for j in range(i,length):
			subStringLen = j - i + 1
			map[subStringLen].append(str[i:j+1])
	return map

"""
Input 'gigxx'. Convert 'gigxx' to 'abacc'

Algorithm:
Iterate over all characters of the input string:
	- If one character has not been changed before (and thus not contained in changedChars), change it to the next available char (a,b,c,d,..)
	- If we have changed the character before, reuse the replacement character.
	
Example:
Input 'gigxx'

First character is 'g'. Change 'g' -> 'a', since it is the first character we are changing, we start with 'a'. All follow up 'g' will be transformed to an 'a'
Next, is 'i'. Since we haven't changed an 'i' before, we change it to 'b'.
Next, is a 'g'. Since we already transformed a 'g' before, we change this one also to 'a'.
Next, is 'x'. Since we haven't changed an 'x' before, we change it to 'c'.
Next, is another 'x'. Since we already transformed an 'x' before, we change this one also to 'c'.

Result: 'abacc'
"""
def normalize(str):
	length = len(str)
	s = list(str)
	char = 'a'
	changedChars = {}
	changedChars[s[0]] = char
	s[0] = char
	
	for i in range(1,length):
		if (s[i] in changedChars):
			s[i] = changedChars[s[i]]
		else:
			char = chr(ord(char)+1)
			changedChars[s[i]] = char
			s[i] = char
		
	return "".join(s)
	
"""
Return true if the normalized version of str is the same as the normalized version of target. False otherwise.
"""	
def isSimilar(str, target):
	strNorm = normalize(str)
	targetNorm = normalize(target)
	return (strNorm == targetNorm)
	
def solve(l,r,s,subStrings):
	target = s[l-1:r]
	subStrCandidates =  subStrings[r-l+1]
	count = 0
	for str in subStrCandidates:
		if (isSimilar(str, target)):
			count +=1
	return count
    
def main():
	f = open("in.txt")
	n, queries = (int(i) for i in f.readline().split())
	s = f.readline().rstrip()
	subStrings = getAllSubstrings(s)
	for q in range(0, queries):
		l, r = (int(i) for i in f.readline().split())        
		print(solve(l,r,s,subStrings))


		
signatures = {}
main()